# Vivino predictor of wine quality based on classification algorithms

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('winequality-red.csv')

# Cleaning and EDA

In [3]:
# Scale the data
scaler = StandardScaler()

In [4]:
X = df
y = X.pop('quality')

In [8]:
Xs = scaler.fit_transform(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(Xs,y,test_size=0.2)

# Model Development

In [11]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train,y_train)
tree_clf.score(X_train,y_train)
#Major overfit - worth using?

1.0

In [12]:
r4_clf = RandomForestClassifier()
r4_clf.fit(X_train,y_train)
r4_clf.score(X_train,y_train)
#again a major overfit - worth using?

1.0

In [13]:
bagg_clf = BaggingClassifier()
bagg_clf.fit(X_train,y_train)
bagg_clf.score(X_train,y_train)
#Sweet - this is a good model.

0.9874902267396404

In [14]:
b00st_clf = AdaBoostClassifier()
b00st_clf.fit(X_train,y_train)
b00st_clf.score(X_train,y_train)
# Meh - not ideal

0.563721657544957

In [15]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train,y_train)
knn_clf.score(X_train,y_train)
# A bit better than the last one but still not so great

0.6966379984362784

In [16]:
log_reg = LogisticRegression(max_iter=1000000)
log_reg.fit(X_train,y_train)
log_reg.score(X_train,y_train)
#Woof

0.5957779515246286

In [29]:
vote_clf = VotingClassifier(estimators=[('lr', log_reg), ('knn', knn_clf),('bagg', bagg_clf)],voting='soft')
vote_clf.fit(X_train,y_train)
vote_clf.score(X_train,y_train)
#Good mix of the models, successful vote.

0.8858483189992181

In [18]:
vote_clf = VotingClassifier(estimators=[('r4', r4_clf), ('tree', tree_clf), ('knn', knn_clf),])
vote_clf.fit(X_train,y_train)
vote_clf.score(X_train,y_train)
# Still overfitting...

1.0